In [6]:
from pyspark.sql import SparkSession

In [7]:
spark = SparkSession\
        .builder\
        .appName("Extraction_Data")\
        .getOrCreate()

24/10/04 20:45:14 WARN Utils: Your hostname, Ubuntu-VirtualBox resolves to a loopback address: 127.0.1.1; using 10.0.2.15 instead (on interface enp0s3)
24/10/04 20:45:14 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/10/04 20:45:15 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [8]:
path = '../../database/origin/ecomerce_dataset.csv'
df = spark.read.option("header", True).csv(path)

In [9]:
path_write = '../../database/dest/bronze/ecommerce_data'
df.write.mode("overwrite").save(path_write, format="parquet")

24/10/04 20:45:28 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


In [10]:
from datetime import datetime

In [11]:
current_year = datetime.now().year
print(current_year)

2024


In [12]:
def generate_url(url_base, endpoint, parameters):
    url = url_base + endpoint
    i = 0
    number_parameters = len(parameters)
    if number_parameters > 0:
        url += '?'
    for parameter in parameters.items():
        url += parameter[0] + '=' + str(parameter[1])
        i += 1
        if i != number_parameters:
            url += '&'
    return url

In [13]:
import requests
import json

pagina = 0
tamanho_pagina = 100
url_base = "https://api.obrasgov.gestao.gov.br"
endpoint = "/obrasgov/api/execucao-financeira"
ano_final = str(current_year)
ano_inicial = str(current_year)

parameters = {
    "tamanhoDaPagina": tamanho_pagina,
    "pagina": pagina,
    "anoFinal": ano_final,
    "anoInicial": ano_inicial
}


url = url_base + endpoint + '?' + 'pagina=' + str(pagina) + '&' + 'tamanhoDaPagina=' + str(tamanho_pagina) + '&' + 'anoFinal=' + str(current_year) + '&' + 'anoInicial=' + str(current_year) 

print(url)

response = requests.request("GET", url)
print(response.status_code)

https://api.obrasgov.gestao.gov.br/obrasgov/api/execucao-financeira?pagina=0&tamanhoDaPagina=100&anoFinal=2024&anoInicial=2024
500


In [14]:
parameters = {
    "tamanhoDaPagina": tamanho_pagina,
    "pagina": pagina,
    "anoFinal": ano_final,
    "anoInicial": ano_inicial
}

generate_url(url_base,endpoint,parameters)

'https://api.obrasgov.gestao.gov.br/obrasgov/api/execucao-financeira?tamanhoDaPagina=100&pagina=0&anoFinal=2024&anoInicial=2024'

In [15]:
len(parameters)

4

In [21]:
import time
import json
from pathlib import Path
import shutil

dest_path = '../../database/dest/bronze/execucao-financeira'
success = False
pagina = 0
errors_consecutives = 0
errors_consecutives_limit = 5
errors = 0
errors_limit = 50
executions = 0
executions_limit = 200
method = "GET"

shutil.rmtree(dest_path)

Path(dest_path).mkdir(parents=True, exist_ok=True)

content_all = []

while success == False and errors_consecutives < errors_consecutives_limit and errors < errors_limit and executions < executions_limit:
    response = requests.request(method, url)
    if response.status_code == 200:
        dest_path_file = dest_path + '/' + str(current_year) + '_' + str(pagina) + '.json'
        pagina += 1
        errors_consecutives = 0
        executions += 1
        content_all += response.json()["content"]
        #with open(dest_path_file, 'w') as f:
        #    json.dump(response.json()["content"],f)
    elif response.status_code == 404:
        success = True 
    else:
        errors_consecutives += 1
        errors += 1
        executions += 1
        if response.status_code == 429:
            time.sleep(1)
    url = url_base + endpoint + '?' + 'pagina=' + str(pagina) + '&' + 'tamanhoDaPagina=' + str(tamanho_pagina) + '&' + 'anoFinal=' + str(current_year) + '&' + 'anoInicial=' + str(current_year) 
    print(f'Status Code: {response.status_code}\n'
           f'Executions: {executions}\n'
           f'Pagina: {pagina}\n' 
           f'N° De registros: {len(content_all)}\n'
           f'Erros: {errors}\n'
           f'Erros Consecutivos: {errors_consecutives}\n')
    time.sleep(1)

if success == True:
    print('Execução Finalizada com sucesso!')
else:
    print('Execução Finalizada com falha!')

dest_path_file = dest_path + '/' + str(ano_inicial) + '_' + str(ano_final) + '.json'

with open(dest_path_file, 'w', encoding='utf-8') as f:
    json.dump(content_all, f, ensure_ascii=False, indent=4)

Status Code: 200
Executions: 1
Pagina: 1
N° De registros: 138
Erros: 0
Erros Consecutivos: 0

Status Code: 200
Executions: 2
Pagina: 2
N° De registros: 272
Erros: 0
Erros Consecutivos: 0

Status Code: 200
Executions: 3
Pagina: 3
N° De registros: 410
Erros: 0
Erros Consecutivos: 0

Status Code: 200
Executions: 4
Pagina: 4
N° De registros: 540
Erros: 0
Erros Consecutivos: 0

Status Code: 200
Executions: 5
Pagina: 5
N° De registros: 673
Erros: 0
Erros Consecutivos: 0

Status Code: 200
Executions: 6
Pagina: 6
N° De registros: 802
Erros: 0
Erros Consecutivos: 0

Status Code: 200
Executions: 7
Pagina: 7
N° De registros: 928
Erros: 0
Erros Consecutivos: 0

Status Code: 200
Executions: 8
Pagina: 8
N° De registros: 1040
Erros: 0
Erros Consecutivos: 0

Status Code: 200
Executions: 9
Pagina: 9
N° De registros: 1142
Erros: 0
Erros Consecutivos: 0

Status Code: 200
Executions: 10
Pagina: 10
N° De registros: 1261
Erros: 0
Erros Consecutivos: 0

Status Code: 200
Executions: 11
Pagina: 11
N° De regist

In [20]:
#response.json()
dest_path_file = dest_path + '/' + str(ano_inicial) + '_' + str(ano_final) + '.json'
dest_path_file

'../../database/dest/bronze/execucao-financeira/2024_2024.json'